In [1]:
import getGmail_Class
import os, sys, logging
from tqdm import trange
from datetime import datetime
import pandas as pd
import re

In [2]:
gGC = getGmail_Class.gmailService()

In [3]:
if not os.path.isdir("file"):
    rootPath = os.path.abspath(os.getcwd())
    os.mkdir("file")

    for key, val in gGC.dict_stock_num2name.items():
        if not os.path.isdir(rootPath + "/file/" + str(key)):
            os.mkdir(rootPath + "/file/" + str(key))

In [7]:
Num, Name, investment_company, Date, Path, ID, recommendResult = [[] for i in range(7)]

# Create log file
if not os.path.isdir("log"):
    os.mkdir("log")

FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level = logging.INFO, filename = "./log/" + datetime.now().strftime("%Y_%m_%d") + '.log', filemode = 'w', format = FORMAT)
logging.info('Updating email start')

# request a list of all the messages
result = gGC.service.users().messages().list(userId = 'me', maxResults = 500, labelIds = ["INBOX"]).execute()
messages = result.get('messages')

# get mail ID from messages
for i in range(len(messages)):
    ID.append(messages[i]['id'])

if len(ID) == 0:
    logging.info('Inbox quantity is 0')
    logging.info('Updating email end')
    sys.exit(0)

# iterate through all the messages
for i in trange(len(ID)):
    # Get the message from its id
    txt = gGC.service.users().messages().get(userId = 'me', id = ID[i]).execute()
    payload = txt['payload']
    headers = payload['headers']
    date = gGC.getDate(headers)
    subject = gGC.getSubject(headers)
    stock_num = gGC.verifySubject(subject)

    if len(stock_num) != 0:
        tempNum, tempName, tempInvestment_company, tempPath = gGC.getResearch_report(subject, stock_num, payload, ID[i])
        temp_recommendResult = gGC.recommend(subject, tempNum)
        recommendResult.extend(temp_recommendResult)
        
        if len(tempNum) != 0:
            for j in range(len(tempNum)):
                Num.append(tempNum[j])
                Name.append(tempName[j])
                Date.append(date)
                investment_company.append(tempInvestment_company)
                Path.append(tempPath[j])
#                     # Modify labels
#                     gGC.modifyLabels(ID[i], "Label2")
    else:
        # Modify labels
#         gGC.modifyLabels(ID[i], "Label3")
        i = i 

df = pd.DataFrame({ "Number" : Num, "Name" : Name, "Investment company" : investment_company, "Date" : Date, "File path" : Path, "Recommend" : recommendResult })

logging.info('Updating email end')

100%|█████████████████████████████████████████| 500/500 [23:14<00:00,  2.79s/it]


In [8]:
df

Number   Name Investment company       Date  \
0     1476     儒鴻               統一投顧  2022_4_11   
1     3357    臺慶科               統一投顧  2022_4_11   
2     9906    欣巴巴               統一投顧  2022_4_11   
3     4991  環宇-KY               統一投顧  2022_4_06   
4     5288  豐祥-KY               統一投顧  2022_4_06   
..     ...    ...                ...        ...   
915   2603     長榮                     2020_7_27   
916   3035     智原                     2020_7_27   
917   5388     中磊                     2020_7_27   
918   6442     光聖                     2020_7_27   
919   9939     宏全                     2020_7_27   

                                        File path Recommend  
0       ./file/1476/統一投顧_儒鴻_統一投顧-20220411投資早報.pdf        買進  
1      ./file/3357/統一投顧_臺慶科_統一投顧-20220411投資早報.pdf        買進  
2      ./file/9906/統一投顧_欣巴巴_統一投顧-20220411投資早報.pdf        買進  
3    ./file/4991/統一投顧_環宇-KY_統一投顧-20220406投資早報.pdf       未評等  
4    ./file/5288/統一投顧_豐祥-KY_統一投顧-20220406投資早報.pdf        買進  
..                                            ...       ...  
915     ./file/2603/長榮_20200727晨會報告(二)-公司訪談摘要.pdf      null  
916     ./file/3035/智原_20200727晨會報告(二)-公司訪談摘要.pdf      null  
917     ./file/5388/中磊_20200727晨會報告(二)-公司訪談摘要.pdf      null  
918     ./file/6442/光聖_20200727晨會報告(二)-公司訪談摘要.pdf      null  
919     ./file/9939/宏全_20200727晨會報告(二)-公司訪談摘要.pdf      null  

[920 rows x 6 columns]